### --- Day 18: Boiling Boulders ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/18

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 18);
WriteLines(inputLines);

Loading puzzle file: Day18.txt
Total lines: 2781
Max line length: 8

4,8,5
10,14,18
8,17,6
15,14,15
8,15,3


In [4]:
string[] testInputLines = 
[
    "2,2,2",
    "1,2,2",
    "3,2,2",
    "2,1,2",
    "2,3,2",
    "2,2,1",
    "2,2,3",
    "2,2,4",
    "2,2,6",
    "1,2,5",
    "3,2,5",
    "2,1,5",
    "2,3,5",
];

Hmm, the first part certainly seems straightforward enough. Each cube adds six units of surface area - for each cube, we check the neighbouring points and subtract the side if a neighbour is present. I suspect the pain is waiting for us in part two :)

In [5]:
using Point3D = (int x, int y, int z);
Point3D AsPoint3D(int[] p) => (p[0], p[1], p[2]);

int SurfaceArea(string[] inputLines)
{
    var points = inputLines.Select(l => l.ParseInts().ToArray());
    var pointSet = points.Select(AsPoint3D).ToHashSet();

    var checkPoints = pointSet.SelectMany(Neighbours);
    var covered = checkPoints.Where(p => pointSet.Contains(p)).Count();

    return pointSet.Count * 6 - covered;
}

IEnumerable<Point3D> Neighbours(Point3D p) 
{
    foreach (var i in List(-1, 1))
    {
        yield return (p.x + i, p.y, p.z);
        yield return (p.x, p.y + i, p.z);
        yield return (p.x, p.y, p.z + i);
    }
}

In [6]:
// In the above example, after counting up all the sides that aren't connected
// to another cube, the total surface area is 64.

var testAnswer = SurfaceArea(testInputLines);
Console.WriteLine(testAnswer);

64


In [7]:
// What is the surface area of your scanned lava droplet?

var part1Answer = SurfaceArea(inputLines);
Console.WriteLine(part1Answer);

4500


In [8]:
// 4500 is correct!
Ensure(4500, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/18

Ok, so for part two, we only need to consider the exterior surfaces of the shape, i.e., we exclude the inside surfaces if the shape is sealed. 

Figuring out of the shape is sealed might be tricky, as we could have an almost-cube with just one piece missing, and the water would be able to ingress. So our trick of determining the inside of a shape by counting edge intersections is not really useful here.

I'm thinking the easiest approach might be to start at a point outside the shape, and search all points in space that are empty, bounded by a box around the shape, effectively "casting" the shape in a mould. Then, we know that every external face of the original shape has a neighbouring point within our external casted shape. If we count these points, it should give us our external surface area.

In [10]:
int ExternalSurfaceArea(string[] inputLines)
{
    var points = inputLines.Select(line => line.ParseInts().ToArray());
    var pointSet = points.Select(AsPoint3D).ToHashSet();

    var (xMin, xMax) = MinMax(pointSet.Select(p => p.x));
    var (yMin, yMax) = MinMax(pointSet.Select(p => p.y));
    var (zMin, zMax) = MinMax(pointSet.Select(p => p.z));

    var (xStart, xEnd) = (xMin - 1, xMax + 1);
    var (yStart, yEnd) = (yMin - 1, yMax + 1);
    var (zStart, zEnd) = (zMin - 1, zMax + 1);

    Point3D startPoint = (xStart, yStart, zStart);

    bool inBounds(Point3D p) => p.x.In(xStart, xEnd) 
                                && p.y.In(yStart, yEnd) 
                                && p.z.In(zStart, zEnd);
    bool isExternal(Point3D p) => inBounds(p) && !pointSet.Contains(p);
    var externalPoints = BFS(startPoint, p => Neighbours(p).Where(isExternal));
    var externalSet = externalPoints.ToHashSet();

    var externalFaces = pointSet.SelectMany(Neighbours)
        .Where(p => externalSet.Contains(p))
        .Count();
    
    return externalFaces;
}

static bool In(this int x, int start, int end) => start <= x && x <= end;

In [11]:
// In the larger example above, exactly one cube of air is trapped within the
// lava droplet (at 2,2,5), so the exterior surface area of the lava droplet is
// 58.

var part2TestAnswer = ExternalSurfaceArea(testInputLines);
Console.WriteLine(part2TestAnswer);

58


In [12]:
// What is the exterior surface area of your scanned lava droplet?

var part2Answer = ExternalSurfaceArea(inputLines);
Console.WriteLine(part2Answer);

2558


In [13]:
// 2558 is correct!
Ensure(2558, part2Answer);